In [ ]:
import pandas as pd

path = r'../chembl_data/chembl_raw.csv'
df = pd.read_csv(path, sep = ';', encoding = 'utf-8')
df.head()

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type,Standard Text Value,Value
0,CHEMBL1645019,NaN,NaN,398.47,0.0,2.85,26,COc1ccc(C(=O)N2CCN(c3ccccn3)CC2)cc1C#Cc1ccccn1,Inhibition,'=',...,CHEMBL1641557,1,Scientific Literature,Bioorg Med Chem Lett,2011.0,NaN,NaN,NaN,NaN,37.0
1,CHEMBL1645022,NaN,NaN,404.50,0.0,2.91,29,COc1ccc(C(=O)N2CCN(c3ccccn3)CC2)cc1C#Cc1nccs1,Inhibition,'=',...,CHEMBL1641557,1,Scientific Literature,Bioorg Med Chem Lett,2011.0,NaN,NaN,NaN,NaN,52.0
2,CHEMBL1645013,NaN,NaN,398.47,0.0,2.85,20,COc1ccc(C(=O)N2CCN(c3cnccn3)CC2)cc1C#Cc1ccccc1,Inhibition,'=',...,CHEMBL1641557,1,Scientific Literature,Bioorg Med Chem Lett,2011.0,NaN,NaN,NaN,NaN,62.0
3,CHEMBL1800667,NaN,NaN,712.80,1.0,4.21,6g,CONC(=O)Nc1ccc(-c2sc3c(c2CN(C)CCN(C)S(C)(=O)=O...,Inhibition,'=',...,CHEMBL1799984,1,Scientific Literature,J Med Chem,2011.0,NaN,NaN,NaN,NaN,17.0
4,CHEMBL1800665,NaN,NaN,672.72,2.0,4.61,6e,CONC(=O)Nc1ccc(-c2sc3c(c2CN(C)CCn2ccnn2)c(=O)n...,Inhibition,'=',...,CHEMBL1799984,1,Scientific Literature,J Med Chem,2011.0,NaN,NaN,NaN,NaN,54.0


In [ ]:
df = df[['Molecule ChEMBL ID', 'Smiles', 'Standard Value', 'Target Name', 'Assay Description', 'Standard Relation']].rename(
    columns = {'Standard Value' : 'Inhibition'}
)

In [ ]:
df.head()

,Molecule ChEMBL ID,Smiles,Inhibition,Target Name,Assay Description,Standard Relation
0,CHEMBL1645019,COc1ccc(C(=O)N2CCN(c3ccccn3)CC2)cc1C#Cc1ccccn1,37.0,Cytochrome P450 3A4,Inhibition of CYP3A4 at 3 uM,'='
1,CHEMBL1645022,COc1ccc(C(=O)N2CCN(c3ccccn3)CC2)cc1C#Cc1nccs1,52.0,Cytochrome P450 3A4,Inhibition of CYP3A4 at 3 uM,'='
2,CHEMBL1645013,COc1ccc(C(=O)N2CCN(c3cnccn3)CC2)cc1C#Cc1ccccc1,62.0,Cytochrome P450 3A4,Inhibition of CYP3A4 at 3 uM,'='
3,CHEMBL1800667,CONC(=O)Nc1ccc(-c2sc3c(c2CN(C)CCN(C)S(C)(=O)=O...,17.0,Cytochrome P450 3A4,Inhibition of human CYP3A4 assessed as convers...,'='
4,CHEMBL1800665,CONC(=O)Nc1ccc(-c2sc3c(c2CN(C)CCn2ccnn2)c(=O)n...,54.0,Cytochrome P450 3A4,Inhibition of human CYP3A4 assessed as convers...,'='


In [ ]:
df_filter = df[
    df['Assay Description'].str.contains('cyp3a4', case = False, na = False) &
    df['Assay Description'].str.contains('10', case = False, na = False) &
    df['Standard Relation'].astype(str).str.contains('=', na = False)
]

In [ ]:
# TDI 관련 데이터를 제외하기 위해 관련 키워드 정의
tdi_keywords = ['TDI', 'time-dependent', 'time dependent', 'preincubation', 'pre-incubation']

# TDI 관련 문구 없는 행만 필터링
df_static = df_filter[~df_filter['Assay Description'].str.contains('|'.join(tdi_keywords), case = False, na = False)]

In [ ]:
df_static.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2032 entries, 3 to 5096
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Molecule ChEMBL ID  2032 non-null   object 
 1   Smiles              2032 non-null   object 
 2   Inhibition          2032 non-null   float64
 3   Target Name         2032 non-null   object 
 4   Assay Description   2032 non-null   object 
 5   Standard Relation   2032 non-null   object 
dtypes: float64(1), object(5)
memory usage: 111.1+ KB


In [ ]:
df_static = df_static[df_static['Inhibition'].astype(float) >= 0]
df_static['Inhibition'] = df_static['Inhibition'].astype(float).clip(upper=100)
df_static = df_static[['Smiles', 'Inhibition']].rename(columns={'Smiles': 'Canonical_Smiles'})

In [ ]:
df_static.head()

,Canonical_Smiles,Inhibition
3,CONC(=O)Nc1ccc(-c2sc3c(c2CN(C)CCN(C)S(C)(=O)=O...,17.00
4,CONC(=O)Nc1ccc(-c2sc3c(c2CN(C)CCn2ccnn2)c(=O)n...,54.00
5,CN(CC(O)COc1ccc2cc(Br)ccc2c1)C1CCCCC1,11.00
7,CN1CCN(Cc2ccc3c(ccn3S(=O)(=O)c3ccccc3)c2)CC1,5.47
8,C[C@@H](C(=O)Nc1nc(C(F)(F)F)co1)c1ccc(OS(=O)(=...,82.00


In [ ]:
df_static.to_csv(r'../processed_data/chembl_final.csv')